<a href="https://colab.research.google.com/github/ankitshripalsingh/Operations-Research/blob/OR/Warehouse%20Location%20Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Warehouse Location Problem

BBasket is an Indian corporation which operates a chain of membership-only grocery clubs. Currently they have thousands of customers in the following locations: Bangalore, Chennai, Hyderabad and Pune. They want to open two warehouses from the given list of locations available to them : Anantapur, Mysore and Vellore. Following is the distance given between warehouse and the customer city. As an analyst, provide them a solution to have minimum total cost of serving all the customers. A customer location can be served by only 1 active warehouse. The table below shows distances between cities in kilometres.

| W / C | Hyderabad | Chennai | Bangalore | Pune |
| :---- | :---- | :---- | :---- | :---- |
|Anantpur|422|482|215|882|
|Mysore|797|482|144|934|
|Vellore|779|157|201|1138|


##  Mathematical formulation / Pyomo components

---

**Sets:** 

To define the indexes for the given problem <br>
- Warehouse location, $w \in W$
- Customer location, $c \in C$

---

**Parameters:**

- Distance between each pair of warehouse-customer location - $\text {d}_{w,c}$ <br>
- Maximum number of warehouses that can be opened - P

---

**Decision variables:**


- Binary/Boolean variable for warehouse supplying to a customer location <br>

> ${x}_{w,c} \ \ where \ \ 0 \leq x_{w,c} \leq 1 \,\,\,\,\,\,\,\,\, \forall c \in C, w \in W $<br>

- Binary/Boolean variable for active warehouse location <br>

> ${y}_{w} \ \ where \ \ y_{w} \in \{0,1\} \,\,\,\,\,\,\,\,\, \forall w \in W $<br>


---


**Objective:**

To *minimize* the sum of distances between the warehouses and customer locations
    
\begin{align}
\textrm {min}\sum \limits _{w,c} \text {d}_{w,c} * \text {x}_{w,c}
\end{align}

---

**Constraints:**

*   One customer location can be satisfied by only one warehouse <br>
>$\sum_{w \in W} x_{w,c} = 1 \,\,\,\,\,\,\,\,\, \forall c \in C$ <br> 

*   Customer location can get goods from only active warehouse <br>

 > $x_{w,c} \leq y_{w} \,\,\,\,\,\,\,\,\, \forall c \in C, w \in W$ <br>

*   Active warehouses should be less than or equal to the required number of warehouses <br>
>$\sum_{w \in W} y_{w} <= P $



### Step1:
<b> Import Pyomo Environment </b>

In [1]:
#Importing Pyomo environment and other required libraries
!pip install -q pyomo
!apt-get install -y -qq coinor-cbc
!apt-get install -y -qq glpk-utils

from pyomo.environ import *

import pandas as pd

     |████████████████████████████████| 9.6 MB 8.0 MB/s 
     |████████████████████████████████| 49 kB 5.2 MB/s 
Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 155320 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unp

### Step2:
<b>Specify / import data</b>

**Sets:** 

To define the indexes for the given problem <br>
- Warehouse location, $w \in W$
- Customer location, $c \in C$

In [2]:
# Indexes - Defining sets/lists containing the indexes for this problem. This is a two index problem - warehouse location 
# and customer location will be the indexes

W = ['Anantpur', 'Mysore', 'Vellore']
C = ['Hyd', 'Chennai', 'Bangalore', 'Pune']

**Parameters:**

- Distance between each pair of warehouse-customer location - $\text {d}_{w,c}$ <br>
- Maximum number of warehouses that can be opened - P

In [4]:
# parameters - distance between each warehouse-customer location
d = {
    (W[0], C[0]) : 422,
    (W[0], C[1]) : 482,
    (W[0], C[2]) : 215,
    (W[0], C[3]) : 882,
    (W[1], C[0]) : 797,
    (W[1], C[1]) : 482,
    (W[1], C[2]) : 144,
    (W[1], C[3]) : 934,
    (W[2], C[0]) : 779,
    (W[2], C[1]) : 157,
    (W[2], C[2]) : 201,
    (W[2], C[3]) : 1138
}

# Maximum number of warehouses that are allowed to be open
P = 2

### Step3:
<b> Create Model Object</b> <br>

In [5]:
# Creating an instance of a Concrete model since we have all the required data before hand
model = ConcreteModel()

### Step4 

<b>Define Decision Variable</b>:


- Binary/Boolean variable for warehouse supplying to a customer location <br>

> ${x}_{w,c} \ \ where \ \ 0 \leq x_{w,c} \leq 1 \,\,\,\,\,\,\,\,\, \forall c \in C, w \in W $<br>

- Binary/Boolean variable for active warehouse location <br>

> ${y}_{w} \ \ where \ \ y_{w} \in \{0,1\} \,\,\,\,\,\,\,\,\, \forall w \in W $<br>

In [6]:
#Define variables
model.x = Var(W, C, bounds = (0,1))
model.y = Var(W, within = Binary)

In [7]:
model.x.pprint()

x : Size=12, Index=x_index
    Key                       : Lower : Value : Upper : Fixed : Stale : Domain
    ('Anantpur', 'Bangalore') :     0 :  None :     1 : False :  True :  Reals
      ('Anantpur', 'Chennai') :     0 :  None :     1 : False :  True :  Reals
          ('Anantpur', 'Hyd') :     0 :  None :     1 : False :  True :  Reals
         ('Anantpur', 'Pune') :     0 :  None :     1 : False :  True :  Reals
      ('Mysore', 'Bangalore') :     0 :  None :     1 : False :  True :  Reals
        ('Mysore', 'Chennai') :     0 :  None :     1 : False :  True :  Reals
            ('Mysore', 'Hyd') :     0 :  None :     1 : False :  True :  Reals
           ('Mysore', 'Pune') :     0 :  None :     1 : False :  True :  Reals
     ('Vellore', 'Bangalore') :     0 :  None :     1 : False :  True :  Reals
       ('Vellore', 'Chennai') :     0 :  None :     1 : False :  True :  Reals
           ('Vellore', 'Hyd') :     0 :  None :     1 : False :  True :  Reals
          ('Vellore', 'Pu

In [8]:
model.y.pprint()

y : Size=3, Index=y_index
    Key      : Lower : Value : Upper : Fixed : Stale : Domain
    Anantpur :     0 :  None :     1 : False :  True : Binary
      Mysore :     0 :  None :     1 : False :  True : Binary
     Vellore :     0 :  None :     1 : False :  True : Binary


### Step5:
<b>Define Objective</b>


To *minimize* the sum of distances between the warehouses and customer locations
    
\begin{align}
\textrm {min}\sum \limits _{w,c} \text {d}_{w,c} * \text {x}_{w,c}
\end{align}

In [10]:
# Minimize the distance between warehouse and customer location
def obj_rule(m):
  return(sum(d[w,c]*model.x[w,c] for w in W for c in C))

model.obj = Objective(rule = obj_rule, sense = minimize)

    'pyomo.core.base.objective.ScalarObjective'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.objective.ScalarObjective'>). This
    is usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().


### Step6:
<b>Define Constraints</b>

*   One customer location can be satisfied by only one warehouse <br>
>$\sum_{w \in W} x_{w,c} = 1 \,\,\,\,\,\,\,\,\, \forall c \in C$ <br> 


In [11]:
# single warehouse per customer
def warehouse_per_cust(model, C):
  return(sum(model.x[w,C] for w in W) == 1)

model.cust_warehouse_one = Constraint(C, rule = warehouse_per_cust)

*   Customer location can get goods from only active warehouse <br>

 > $x_{w,c} \leq y_{w} \,\,\,\,\,\,\,\,\, \forall c \in C, w \in W$ <br>


In [12]:
#only active warehouse can serve a customer location
def active_warehouse(model, w, c):
  return(model.x[w,c] <= model.y[w])

model.is_active_warehouse = Constraint(W, C, rule = active_warehouse)

In [14]:
model.is_active_warehouse.pprint()

is_active_warehouse : Size=12, Index=is_active_warehouse_index, Active=True
    Key                       : Lower : Body                                : Upper : Active
    ('Anantpur', 'Bangalore') :  -Inf : x[Anantpur,Bangalore] - y[Anantpur] :   0.0 :   True
      ('Anantpur', 'Chennai') :  -Inf :   x[Anantpur,Chennai] - y[Anantpur] :   0.0 :   True
          ('Anantpur', 'Hyd') :  -Inf :       x[Anantpur,Hyd] - y[Anantpur] :   0.0 :   True
         ('Anantpur', 'Pune') :  -Inf :      x[Anantpur,Pune] - y[Anantpur] :   0.0 :   True
      ('Mysore', 'Bangalore') :  -Inf :     x[Mysore,Bangalore] - y[Mysore] :   0.0 :   True
        ('Mysore', 'Chennai') :  -Inf :       x[Mysore,Chennai] - y[Mysore] :   0.0 :   True
            ('Mysore', 'Hyd') :  -Inf :           x[Mysore,Hyd] - y[Mysore] :   0.0 :   True
           ('Mysore', 'Pune') :  -Inf :          x[Mysore,Pune] - y[Mysore] :   0.0 :   True
     ('Vellore', 'Bangalore') :  -Inf :   x[Vellore,Bangalore] - y[Vellore] :   0.0 :  

*   Active warehouses should be less than or equal to the required number of warehouses <br>
>$\sum_{w \in W} y_{w} <= P $

In [13]:
# number of warehouses can't be more than what is required
def max_warehouses(model):
  return(sum(model.y[w] for w in W) <= P)

model.max_active_warehouses = Constraint(rule = max_warehouses)

### Step7:
<b>Create solver & solve model </b><br>

In [15]:
result = SolverFactory('glpk').solve(model)
result.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1662.0
  Upper bound: 1662.0
  Number of objectives: 1
  Number of constraints: 18
  Number of variables: 16
  Number of nonzeros: 40
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.019756555557250977
# ----------------------------------------------------------
#   Solution Information
# -------------------------------

In [16]:
model.pprint()

8 Set Declarations
    cust_warehouse_one_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'Hyd', 'Chennai', 'Bangalore', 'Pune'}
    is_active_warehouse_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain                                                  : Size : Members
        None :     2 : is_active_warehouse_index_0*is_active_warehouse_index_1 :   12 : {('Anantpur', 'Hyd'), ('Anantpur', 'Chennai'), ('Anantpur', 'Bangalore'), ('Anantpur', 'Pune'), ('Mysore', 'Hyd'), ('Mysore', 'Chennai'), ('Mysore', 'Bangalore'), ('Mysore', 'Pune'), ('Vellore', 'Hyd'), ('Vellore', 'Chennai'), ('Vellore', 'Bangalore'), ('Vellore', 'Pune')}
    is_active_warehouse_index_0 : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'Anantpur', 'Mysore', 'Vellore'}
    is_active_warehouse_index_1 : Size=1, Index=None, Ordered=Insertion

### Step8:
<b>Display Results </b>

In [17]:
output = []

for w in W:
  for c in C:
    active = model.x[w,c].value
    distance = d[w,c]
    output.append([w, c, active, active*distance])

print(output)

[['Anantpur', 'Hyd', 1.0, 422.0], ['Anantpur', 'Chennai', 0.0, 0.0], ['Anantpur', 'Bangalore', 0.0, 0.0], ['Anantpur', 'Pune', 1.0, 882.0], ['Mysore', 'Hyd', 0.0, 0.0], ['Mysore', 'Chennai', 0.0, 0.0], ['Mysore', 'Bangalore', 0.0, 0.0], ['Mysore', 'Pune', 0.0, 0.0], ['Vellore', 'Hyd', 0.0, 0.0], ['Vellore', 'Chennai', 1.0, 157.0], ['Vellore', 'Bangalore', 1.0, 201.0], ['Vellore', 'Pune', 0.0, 0.0]]


In [18]:
pd.DataFrame(output, columns = ['Warehouse', 'Customer', 'Is_Active', 'Distance']).pivot(index = 'Warehouse', columns = 'Customer', values = 'Is_Active')

Customer,Bangalore,Chennai,Hyd,Pune
Warehouse,,,,
Anantpur,0.0,0.0,1.0,1.0
Mysore,0.0,0.0,0.0,0.0
Vellore,1.0,1.0,0.0,0.0


In [19]:
pd.DataFrame(output, columns = ['Warehouse', 'Customer', 'Is_Active', 'Distance']).pivot(index = 'Warehouse', columns = 'Customer', values = 'Distance')

Customer,Bangalore,Chennai,Hyd,Pune
Warehouse,,,,
Anantpur,0.0,0.0,422.0,882.0
Mysore,0.0,0.0,0.0,0.0
Vellore,201.0,157.0,0.0,0.0


In [20]:
model.display()

Model unknown

  Variables:
    x : Size=12, Index=x_index
        Key                       : Lower : Value : Upper : Fixed : Stale : Domain
        ('Anantpur', 'Bangalore') :     0 :   0.0 :     1 : False : False :  Reals
          ('Anantpur', 'Chennai') :     0 :   0.0 :     1 : False : False :  Reals
              ('Anantpur', 'Hyd') :     0 :   1.0 :     1 : False : False :  Reals
             ('Anantpur', 'Pune') :     0 :   1.0 :     1 : False : False :  Reals
          ('Mysore', 'Bangalore') :     0 :   0.0 :     1 : False : False :  Reals
            ('Mysore', 'Chennai') :     0 :   0.0 :     1 : False : False :  Reals
                ('Mysore', 'Hyd') :     0 :   0.0 :     1 : False : False :  Reals
               ('Mysore', 'Pune') :     0 :   0.0 :     1 : False : False :  Reals
         ('Vellore', 'Bangalore') :     0 :   1.0 :     1 : False : False :  Reals
           ('Vellore', 'Chennai') :     0 :   1.0 :     1 : False : False :  Reals
               ('Vellore', '